In [249]:

import pandas as pd 
import numpy as np
import re
from summarytools import dfSummary


## ETL Extracción, transformación y carga

# `homicidios - hechos`

In [250]:
homicidios_hechos = pd.read_excel("Datasets/raw/homicidios.xlsx", sheet_name="HECHOS")
homicidios_victimas = pd.read_excel("Datasets/raw/homicidios.xlsx", sheet_name="VICTIMAS")
comunas= pd.read_excel("Datasets/extra/comunas.xlsx")


In [251]:
homicidios_hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO


In [252]:
homicidios_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Examen de nulos y duplicados

In [253]:
homicidios_hechos.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

- Se analizan las columnas con datos nulos

In [254]:
homicidios_hechos[["Altura", "Calle", "Dirección Normalizada", "Cruce"]].head(5)

,Altura,Calle,Dirección Normalizada,Cruce
0,NaN,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...","FERNANDEZ DE LA CRUZ, F., GRAL. AV."
1,NaN,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",DE LOS CORRALES AV.
2,2034.0,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,NaN
3,NaN,LARRAZABAL AV.,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.","VILLEGAS, CONRADO, GRAL."
4,NaN,SAN JUAN AV.,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.","SAENZ PE?A, LUIS, PRES."


In [255]:
homicidios_hechos[homicidios_hechos["Dirección Normalizada"].isnull()][["Dirección Normalizada", "Cruce","Altura", "Calle"]]

,Dirección Normalizada,Cruce,Altura,Calle
38,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
106,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA
119,NaN,NaN,NaN,NaN
180,NaN,NaN,NaN,AUTOPISTA PERITO MORENO
181,NaN,NaN,NaN,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.
313,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
546,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
621,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA


Tras examinar datos nulos, que representan el 5,1% del dataset:
- **`Calle`** tiene un solo dato nulo.
- **`Dirección Normalizada`** 8 nulos. 
- **`Cruce`** tiene 171 nulos.
- **`Altura`** tiene 567 nulos.

Analizando los datos de la `Altura` se procede a eliminarla debido a la gran cantidad de faltantes y no existiendo manera de poder rellenar las colummas; ya que altura se refiere a la numeración de la calle donde se produce el siniestro y la mayoria de los casos se producen en cruces o esquinas.
La columna `Cruce` solo va a tener dato cuando el hecho se porduce en un cruce de calles, por lo que se conserva de esta manera.
La columna `Dirección Normalizada` no tiene posibilidad de completarse con las columnas del dataset; pero contiene solo 1,1% de nulos por lo se conserva de esta manera.

Se elimina la columna "Altura"

In [256]:
homicidios_hechos= homicidios_hechos.drop("Altura", axis=1)
homicidios_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

Duplicados

In [257]:
homicidios_hechos.duplicated().sum()

0

Se transforman los datos de la columna 'HH' a tipo numérico.

In [258]:
homicidios_hechos["HH"].unique()

array([4, 1, 7, 0, 5, 18, 19, 15, 11, 22, 16, 9, 23, 6, 10, 17, 12, 8, 20,
       21, 14, 3, 2, 13, 'SD'], dtype=object)

In [259]:
# Cambio el dato horas  a tipo de dato numerico y luego a entero 
homicidios_hechos["HH"] = pd.to_numeric(homicidios_hechos ["HH"], errors="coerce")
homicidios_hechos["HH"] = homicidios_hechos["HH"].astype("Int64")

In [260]:
homicidios_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     695 non-null    Int64         
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Cruce                  525 non-null    object        
 12  Dirección Normalizada  688 non-null    object        
 13  COMUN

Transformacion a columnas 'pos x' y 'pos y'

In [261]:
# Filtrar filas con posiciones no válidas
valid_positions = ~(homicidios_hechos['pos x'].str.contains('[^0-9.,-]') | homicidios_hechos['pos y'].str.contains('[^0-9.,-]') | (homicidios_hechos['pos x'] == '.') | (homicidios_hechos['pos y'] == '.'))

# Establecer posiciones no válidas en NaN
homicidios_hechos.loc[~valid_positions, ['pos x', 'pos y']] = np.nan

# Convertir las cadenas a tipo numérico (float)
homicidios_hechos['pos x'] = homicidios_hechos['pos x'].str.replace(',', '.').astype(float)
homicidios_hechos['pos y'] = homicidios_hechos['pos y'].str.replace(',', '.').astype(float)

In [262]:
homicidios_hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO


Columnas resultantes

In [263]:
homicidios_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

En la columna 'Comuna' tenemos una numeracion del 1 al 15. Esta numeración identifica a cada una de las divisiones comunales de la ciudad autónoma de Buenos Aires.

Para agregar valor al dataset, agregaremos la descripción de cada una de las comunas, con los Barrios que la conforman.

Para eso utilizaremos el dataset 'comunas' importado junto al dataset `homicidios`. Este dataset fue descargado del siguiente sitio web: https://buenosaires.gob.ar/comunas

In [264]:
#Se realiza el merge a través de la columna en comun `Comuna` y nos va a quedar anexada la columna `Barrios`

homicidios_hechos=homicidios_hechos.merge(comunas, on="COMUNA", how="left")

#Modifico las mayúsculas por minúsculas
homicidios_hechos=homicidios_hechos.rename(columns={
    "BARRIOS": "Barrios"
})


In [265]:
homicidios_hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,Barrios
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,LINIERS - MATADEROS - PARQUE AVELLANEDA
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,CONSTITUCION - MONTSERRAT - PUERTO MADERO - RE...


Las colummas tienen nombres en mayúsculas y también en minúsculas. Se estandarizan a modo de buena práctica

In [266]:
#Cambio la primer letra a mayúscula
homicidios_hechos.columns = [x.capitalize() for x in homicidios_hechos.columns]
# Reemplazo guiones por espacios
homicidios_hechos.columns = homicidios_hechos.columns.str.replace("_", " ")
#Renombra columnas 
homicidios_hechos= homicidios_hechos.rename(columns={"N victimas": "Num víctimas",
                                            "Aaaa": "Año",
                                            "Mm":"Mes",
                                            "Dd": "Día",
                                            "Hora": "Hora completa",
                                            "Hh": "Hora"}) 

homicidios_hechos.columns

Index(['Id', 'Num víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora completa',
       'Hora', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Barrios'],
      dtype='object')

Al analizar los datos de la columna `Día` se muestra solo el numero del día y no el nombre correspondiente por lo que se crea la columna `Día semana` donde se corresponde cada fecha con un dia de la semana para poder analizar luego como se relacionan los siniestros con los dias de la semana

In [267]:
homicidios_hechos.Día

0       1
1       2
2       3
3      10
4      21
       ..
691    13
692    20
693    30
694    15
695    18
Name: Día, Length: 696, dtype: int64

In [268]:
#Creo una columna nueva `Día semana` que contenga el nombre del día de la semana
homicidios_hechos['Día semana'] = homicidios_hechos['Fecha'].dt.strftime('%A')

#Creo un diccionario para modificar a español los nombres de los dás de la semana y lo aplico a la columna nueva
dic={  'Friday':'Viernes 6',
       'Saturday': 'Sabado 7',
       'Sunday': 'Domingo 1',  
       'Thursday': 'Jueves 5',
        'Monday': 'Lunes 2',
        'Wednesday': 'Miercoles 4',
       'Tuesday': 'Martes 3'}
homicidios_hechos['Día semana'] =homicidios_hechos['Día semana'].replace(dic)

In [269]:
homicidios_hechos.columns

Index(['Id', 'Num víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora completa',
       'Hora', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Barrios', 'Día semana'],
      dtype='object')

La colmuna `Comuna` se modifica agregando el termino Comuna antes del número, a fin de lograr una variable categórica que me simplifique el análisis

In [270]:
homicidios_hechos["Comuna"].unique()

array([ 8,  9,  1, 11, 15,  4,  7, 12,  3, 13, 14, 10,  6,  2,  5,  0],
      dtype=int64)

In [271]:
#Agrego la palabra Comuna delante del dato numérico
#se aplica la función lambda a cada fila del dataframe 
homicidios_hechos["Comuna"] = homicidios_hechos.apply(lambda x: "Comuna "+str(x["Comuna"]), axis=1)


In [272]:
homicidios_hechos["Comuna"].unique()

array(['Comuna 8', 'Comuna 9', 'Comuna 1', 'Comuna 11', 'Comuna 15',
       'Comuna 4', 'Comuna 7', 'Comuna 12', 'Comuna 3', 'Comuna 13',
       'Comuna 14', 'Comuna 10', 'Comuna 6', 'Comuna 2', 'Comuna 5',
       'Comuna 0'], dtype=object)

In [273]:
homicidios_hechos.isna().sum()

Id                         0
Num víctimas               0
Fecha                      0
Año                        0
Mes                        0
Día                        0
Hora completa              0
Hora                       1
Lugar del hecho            0
Tipo de calle              0
Calle                      1
Cruce                    171
Dirección normalizada      8
Comuna                     0
Xy (caba)                  0
Pos x                     12
Pos y                     12
Participantes              0
Victima                    0
Acusado                    0
Barrios                    2
Día semana                 0
dtype: int64

In [274]:
homicidios_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     696 non-null    object        
 1   Num víctimas           696 non-null    int64         
 2   Fecha                  696 non-null    datetime64[ns]
 3   Año                    696 non-null    int64         
 4   Mes                    696 non-null    int64         
 5   Día                    696 non-null    int64         
 6   Hora completa          696 non-null    object        
 7   Hora                   695 non-null    Int64         
 8   Lugar del hecho        696 non-null    object        
 9   Tipo de calle          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Cruce                  525 non-null    object        
 12  Dirección normalizada  688 non-null    object        
 13  Comun

# `homicidios- víctimas`

In [275]:
homicidios_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


### Transformación de los datos

- Conversión del tipo de dato

In [276]:
#Cambio la columna a tipo datetime
homicidios_victimas["FECHA_FALLECIMIENTO"] = pd.to_datetime(homicidios_victimas["FECHA_FALLECIMIENTO"], format='%Y-%m-%d', errors='coerce')

In [277]:
#Una vez que se paso el tipo de dato a datetime que es una fecha y hora completa se utiliza este codigo para que solo se quede en formato fecha sin hora
homicidios_victimas["FECHA_FALLECIMIENTO"] = homicidios_victimas["FECHA_FALLECIMIENTO"].dt.date

#Luego se reemplazan las fechas en cero con espacios vacios a fin de su manipulación
homicidios_victimas["FECHA_FALLECIMIENTO"] = homicidios_victimas ["FECHA_FALLECIMIENTO"].replace(pd.NaT,"")

In [278]:
#Cambio la columna edad a numerico y luego a tipo entero
homicidios_victimas ["EDAD"] = pd.to_numeric(homicidios_victimas["EDAD"], errors="coerce")
homicidios_victimas ["EDAD"] = homicidios_victimas["EDAD"].astype("Int64")

In [279]:
#Utilizo Numpy para reemplazar por NaN los sin datos o vacios y luego paso a float para que cuando quiera aplicar estadisticas no tenga errores debido a esos datos
homicidios_victimas["EDAD"] = homicidios_victimas["EDAD"].replace([" ","SD"], np.nan)

In [280]:
homicidios_victimas["EDAD"] = homicidios_victimas["EDAD"].fillna(0.0)

In [281]:
#Controlo los datos de la columna
homicidios_victimas ["EDAD"].unique()

<IntegerArray>
[19, 70, 30, 18, 29, 22, 16, 59, 65, 34, 41, 50, 38, 21, 52, 36, 20, 54,  0,
 56, 24, 78, 79, 26, 57, 37, 58, 23, 60, 42, 53, 51, 40, 87, 76, 75, 35, 80,
 43, 45, 67, 27, 55, 49, 81, 25, 33, 46, 83, 39, 28,  7, 48,  4, 82, 32, 17,
 47, 61, 10, 95, 73, 84, 66, 85,  1, 15, 13, 77, 44, 31, 62, 74, 71, 11, 86,
 69, 72, 12, 63, 92, 68, 91, 64,  5, 88]
Length: 86, dtype: Int64

In [282]:
homicidios_victimas.head(3)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03


No se encuentran datos nulos, ni campos vacios

In [283]:
homicidios_victimas.isnull().sum()

ID_hecho               0
FECHA                  0
AAAA                   0
MM                     0
DD                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
dtype: int64

Se crea la columna `Rango etario` basada en la columna `Edad`, para mejorar el análisis de los datos 

In [284]:
# Se utiliza la función `pd.cut()`, que permite agrupar valores numéricos en intervalos o categorías


# Definir los límites de los rangos etarios y las etiquetas
bins = [0, 20, 40, 60, 80, 100]
labels = ['Menos de 20', ' Entre 21- 40', 'Entre 41-60', 'Entre 61-80', 'Más de 80']

# Utilizar pd.cut() para crear la columna de rango etario
homicidios_victimas['Rango_etario'] = pd.cut(homicidios_victimas['EDAD'], bins=bins, labels=labels, right=False)

homicidios_victimas.head(3)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,Rango_etario
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01,Menos de 20
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02,Entre 61-80
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03,Entre 21- 40


A continuación se estandarizan las mayúsculas y minúsculas del dataset

In [285]:
# Se coloca la primera en mayúscula
homicidios_victimas.columns = [x.capitalize() for x in homicidios_victimas.columns]
# Se reemplazan los guiones por espacios
homicidios_victimas.columns = homicidios_victimas.columns.str.replace('_', ' ')
# Se renombran algunas columnas
homicidios_victimas = homicidios_victimas.rename(columns={'Id hecho': 'Id',
                                                      'Aaaa':'Año',
                                                      'Mm':'Mes',
                                                      'Dd':'Día',
                                                      'Victima':'Víctima'})
homicidios_victimas.columns

Index(['Id', 'Fecha', 'Año', 'Mes', 'Día', 'Rol', 'Víctima', 'Sexo', 'Edad',
       'Fecha fallecimiento', 'Rango etario'],
      dtype='object')

A partir de los datos, se analiza eliminar columnas que se repiten en ambos dataset para realizar luego la unión y trabajar con un conjunto de datos.
Se proceden a eliminar del dataset `homicidios_victimas` las columnas `Fecha`, `Año`, `Mes`, `Día` y `Victima` que contienen la misma información que se encuentra en `homicidios_hechos`. 

In [286]:
columnas_a_borrar = ['Fecha', 'Año', 'Mes', 'Día', 'Víctima']

homicidios_victimas = homicidios_victimas.drop(columns=columnas_a_borrar)


In [287]:
homicidios_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Id                   717 non-null    object  
 1   Rol                  717 non-null    object  
 2   Sexo                 717 non-null    object  
 3   Edad                 717 non-null    Int64   
 4   Fecha fallecimiento  717 non-null    object  
 5   Rango etario         717 non-null    category
dtypes: Int64(1), category(1), object(4)
memory usage: 29.7+ KB


## Unificación de los dataset

Por medio de la función merge se unifican los dataset transformados anteriormente

In [288]:
siniestros= homicidios_victimas.merge(homicidios_hechos, on="Id", how='left')
siniestros.columns

Index(['Id', 'Rol', 'Sexo', 'Edad', 'Fecha fallecimiento', 'Rango etario',
       'Num víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora completa', 'Hora',
       'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Barrios', 'Día semana'],
      dtype='object')

In [289]:
siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     717 non-null    object        
 1   Rol                    717 non-null    object        
 2   Sexo                   717 non-null    object        
 3   Edad                   717 non-null    Int64         
 4   Fecha fallecimiento    717 non-null    object        
 5   Rango etario           717 non-null    category      
 6   Num víctimas           717 non-null    int64         
 7   Fecha                  717 non-null    datetime64[ns]
 8   Año                    717 non-null    int64         
 9   Mes                    717 non-null    int64         
 10  Día                    717 non-null    int64         
 11  Hora completa          717 non-null    object        
 12  Hora                   716 non-null    Int64         
 13  Lugar

Al unificar los dataset se encontraron datos cambiados a tipo float que son necesarios como enteros, por lo que se transforman

In [290]:
siniestros["Num víctimas"].unique()

array([1, 2, 3], dtype=int64)

In [291]:
siniestros ["Num víctimas"] = siniestros ["Num víctimas"].fillna(0.0)
siniestros ["Num víctimas"] = siniestros ["Num víctimas"].astype ("Int64")

In [292]:
siniestros["Num víctimas"].unique()

<IntegerArray>
[1, 2, 3]
Length: 3, dtype: Int64

In [293]:
siniestros["Año"].unique()

array([2016, 2017, 2018, 2019, 2020, 2021], dtype=int64)

In [294]:
siniestros ["Año"]= siniestros ["Año"].astype ("Int64")
siniestros["Año"].unique()

<IntegerArray>
[2016, 2017, 2018, 2019, 2020, 2021]
Length: 6, dtype: Int64

In [295]:
siniestros ["Mes"]= siniestros ["Mes"].astype ("Int64")
siniestros["Mes"].unique()

<IntegerArray>
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Length: 12, dtype: Int64

In [296]:
siniestros ["Día"]= siniestros ["Día"].astype ("Int64")
siniestros["Día"].unique()

<IntegerArray>
[ 1,  2,  3, 10, 21, 24, 29,  8, 14, 15, 17, 28,  4, 12, 13, 19, 23, 30, 31,
 11, 20, 22, 25, 26, 16, 18, 27,  7,  9,  5,  6]
Length: 31, dtype: Int64

In [297]:
siniestros.head(3)

,Id,Rol,Sexo,Edad,Fecha fallecimiento,Rango etario,Num víctimas,Fecha,Año,Mes,...,Dirección normalizada,Comuna,Xy (caba),Pos x,Pos y,Participantes,Victima,Acusado,Barrios,Día semana
0,2016-0001,CONDUCTOR,MASCULINO,19,2016-01-01,Menos de 20,1,2016-01-01,2016,1,...,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",Comuna 8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI,Viernes 6
1,2016-0002,CONDUCTOR,MASCULINO,70,2016-01-02,Entre 61-80,1,2016-01-02,2016,1,...,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",Comuna 9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,LINIERS - MATADEROS - PARQUE AVELLANEDA,Sabado 7
2,2016-0003,CONDUCTOR,MASCULINO,30,2016-01-03,Entre 21- 40,1,2016-01-03,2016,1,...,ENTRE RIOS AV. 2034,Comuna 1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,CONSTITUCION - MONTSERRAT - PUERTO MADERO - RE...,Domingo 1


Se realiza un resumen descriptivo de los datos de `siniestros` 

In [298]:
siniestros.describe()

,Edad,Num víctimas,Fecha,Año,Mes,Día,Hora,Pos x,Pos y
count,717.0,717.0,717,717.0,717.0,717.0,716.0,704.000000,704.000000
mean,39.050209,1.061367,2018-09-09 01:24:21.087866112,2018.174338,6.688982,15.953975,11.554469,-58.441664,-34.619700
min,0.0,1.0,2016-01-01 00:00:00,2016.0,1.0,1.0,0.0,-58.529942,-34.705250
25%,24.0,1.0,2017-04-01 00:00:00,2017.0,4.0,9.0,6.0,-58.476556,-34.643918
50%,35.0,1.0,2018-07-07 00:00:00,2018.0,7.0,16.0,11.0,-58.444513,-34.622866
75%,55.0,1.0,2019-12-18 00:00:00,2019.0,10.0,23.0,17.0,-58.401841,-34.597013
max,95.0,3.0,2021-12-30 00:00:00,2021.0,12.0,31.0,23.0,-58.356082,-34.534654
std,22.010669,0.257024,NaN,1.672655,3.57514,8.674876,6.700055,0.046397,0.035292


In [299]:
siniestros.columns

Index(['Id', 'Rol', 'Sexo', 'Edad', 'Fecha fallecimiento', 'Rango etario',
       'Num víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora completa', 'Hora',
       'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Barrios', 'Día semana'],
      dtype='object')

Antes de realizar en EDA propiamiente dicho se ordenan las columnas

In [300]:
#Creo una lista con el orden 
nuevo_orden=["Id", "Fecha", "Año", "Mes", "Día","Día semana", "Hora completa", "Hora", "Num víctimas", "Participantes", "Rol", "Acusado", "Victima", "Sexo", "Edad", "Rango etario", "Fecha fallecimiento", "Lugar del hecho", "Tipo de calle", "Calle", "Cruce", "Dirección normalizada", "Comuna", "Xy (caba)", "Pos x", "Pos y"]

#Reorganizo el dataframe untilizando la lista y asignandola nuevamente aldataframe siniestros
siniestros_limpio = siniestros[nuevo_orden]
siniestros_limpio.columns



Index(['Id', 'Fecha', 'Año', 'Mes', 'Día', 'Día semana', 'Hora completa',
       'Hora', 'Num víctimas', 'Participantes', 'Rol', 'Acusado', 'Victima',
       'Sexo', 'Edad', 'Rango etario', 'Fecha fallecimiento',
       'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y'],
      dtype='object')

In [301]:
siniestros_limpio.head(3)

,Id,Fecha,Año,Mes,Día,Día semana,Hora completa,Hora,Num víctimas,Participantes,...,Fecha fallecimiento,Lugar del hecho,Tipo de calle,Calle,Cruce,Dirección normalizada,Comuna,Xy (caba),Pos x,Pos y
0,2016-0001,2016-01-01,2016,1,1,Viernes 6,04:00:00,4,1,MOTO-AUTO,...,2016-01-01,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",Comuna 8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570
1,2016-0002,2016-01-02,2016,1,2,Sabado 7,01:15:00,1,1,AUTO-PASAJEROS,...,2016-01-02,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",Comuna 9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777
2,2016-0003,2016-01-03,2016,1,3,Domingo 1,07:00:00,7,1,MOTO-AUTO,...,2016-01-03,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,Comuna 1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894


In [302]:
siniestros_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     717 non-null    object        
 1   Fecha                  717 non-null    datetime64[ns]
 2   Año                    717 non-null    Int64         
 3   Mes                    717 non-null    Int64         
 4   Día                    717 non-null    Int64         
 5   Día semana             717 non-null    object        
 6   Hora completa          717 non-null    object        
 7   Hora                   716 non-null    Int64         
 8   Num víctimas           717 non-null    Int64         
 9   Participantes          717 non-null    object        
 10  Rol                    717 non-null    object        
 11  Acusado                717 non-null    object        
 12  Victima                717 non-null    object        
 13  Sexo 

In [ ]:
siniestros_limpio["Fecha"]=siniestros_limpio["Fecha"].dt.date

In [304]:
siniestros_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Id                     717 non-null    object  
 1   Fecha                  717 non-null    object  
 2   Año                    717 non-null    Int64   
 3   Mes                    717 non-null    Int64   
 4   Día                    717 non-null    Int64   
 5   Día semana             717 non-null    object  
 6   Hora completa          717 non-null    object  
 7   Hora                   716 non-null    Int64   
 8   Num víctimas           717 non-null    Int64   
 9   Participantes          717 non-null    object  
 10  Rol                    717 non-null    object  
 11  Acusado                717 non-null    object  
 12  Victima                717 non-null    object  
 13  Sexo                   717 non-null    object  
 14  Edad                   717 non-null    Int

Se guarda el archivo como csv para optimizar su manipulación

In [305]:
siniestros_limpio.to_csv("Datasets/clean/siniestos_limpio_2.csv", index=False)

In [306]:
siniestros_limpio.columns

Index(['Id', 'Fecha', 'Año', 'Mes', 'Día', 'Día semana', 'Hora completa',
       'Hora', 'Num víctimas', 'Participantes', 'Rol', 'Acusado', 'Victima',
       'Sexo', 'Edad', 'Rango etario', 'Fecha fallecimiento',
       'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y'],
      dtype='object')

In [307]:
siniestros_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   Id                     717 non-null    object  
 1   Fecha                  717 non-null    object  
 2   Año                    717 non-null    Int64   
 3   Mes                    717 non-null    Int64   
 4   Día                    717 non-null    Int64   
 5   Día semana             717 non-null    object  
 6   Hora completa          717 non-null    object  
 7   Hora                   716 non-null    Int64   
 8   Num víctimas           717 non-null    Int64   
 9   Participantes          717 non-null    object  
 10  Rol                    717 non-null    object  
 11  Acusado                717 non-null    object  
 12  Victima                717 non-null    object  
 13  Sexo                   717 non-null    object  
 14  Edad                   717 non-null    Int